In [0]:
# Databricks Notebook: Gold Summary for Futures Positions Dashboard
from pyspark.sql import functions as F
from pyspark.sql.window import Window

CATALOG = "demo_catalog"
SCHEMA  = "demo_schema"
SILVER  = f"{CATALOG}.{SCHEMA}.silver_futures_positions"
GOLD    = f"{CATALOG}.{SCHEMA}.gold_futures_positions_summary"

# =========================
# (A) Gold 테이블 생성
# =========================
spark.sql(f"""
CREATE TABLE IF NOT EXISTS {GOLD} (
  uid        STRING,
  symbol     STRING,
  entryPrice DOUBLE,
  markPrice  DOUBLE,
  pnl        DOUBLE,
  roe        DOUBLE,
  amount     DOUBLE,
  leverage   DOUBLE,
  event_time TIMESTAMP,
  updated_at TIMESTAMP
) USING DELTA
""")

# =========================
# (B) 최신 포지션만 추출
# =========================
silver = spark.table(SILVER)

w = Window.partitionBy("uid","symbol").orderBy(F.col("event_time").desc())

latest_positions = (
    silver
      .withColumn("rn", F.row_number().over(w))
      .where("rn = 1")
      .drop("rn")
      .withColumn("updated_at", F.current_timestamp())
)

# =========================
# (C) 저장 (overwrite)
# =========================
latest_positions.write.mode("overwrite").option("mergeSchema", "true").saveAsTable(GOLD)

print(f"[SUMMARY] refreshed: {GOLD}")
